In [ ]:
# 1. 导入库及读取数据
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import warnings

warnings.filterwarnings('ignore')
# 解决中文显示问题
plt.rcParams['font.sans-serif'] = ['SimHei']  # Windows系统黑体
plt.rcParams['axes.unicode_minus'] = False    # 修复负号显示

df = pd.read_csv("US-pumpkins.csv")  # 以pandas库的read_csv函数读取csv文件
df.head()  # 查看前5行数据

,City Name,Type,Package,Variety,Sub Variety,Grade,Date,Low Price,High Price,Mostly Low,...,Unit of Sale,Quality,Condition,Appearance,Storage,Crop,Repack,Trans Mode,Unnamed: 24,Unnamed: 25
0,BALTIMORE,NaN,24 inch bins,NaN,NaN,NaN,4/29/17,270.0,280.0,270.0,...,NaN,NaN,NaN,NaN,NaN,NaN,E,NaN,NaN,NaN
1,BALTIMORE,NaN,24 inch bins,NaN,NaN,NaN,5/6/17,270.0,280.0,270.0,...,NaN,NaN,NaN,NaN,NaN,NaN,E,NaN,NaN,NaN
2,BALTIMORE,NaN,24 inch bins,HOWDEN TYPE,NaN,NaN,9/24/16,160.0,160.0,160.0,...,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN
3,BALTIMORE,NaN,24 inch bins,HOWDEN TYPE,NaN,NaN,9/24/16,160.0,160.0,160.0,...,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN
4,BALTIMORE,NaN,24 inch bins,HOWDEN TYPE,NaN,NaN,11/5/16,90.0,100.0,90.0,...,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN


In [7]:
# 2. 数据基本信息
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1757 entries, 0 to 1756
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   City Name        1757 non-null   object 
 1   Type             45 non-null     object 
 2   Package          1757 non-null   object 
 3   Variety          1752 non-null   object 
 4   Sub Variety      296 non-null    object 
 5   Grade            0 non-null      float64
 6   Date             1757 non-null   object 
 7   Low Price        1757 non-null   float64
 8   High Price       1757 non-null   float64
 9   Mostly Low       1654 non-null   float64
 10  Mostly High      1654 non-null   float64
 11  Origin           1754 non-null   object 
 12  Origin District  131 non-null    object 
 13  Item Size        1478 non-null   object 
 14  Color            1141 non-null   object 
 15  Environment      0 non-null      float64
 16  Unit of Sale     162 non-null    object 
 17  Quality       

In [8]:
# 3.缺失值检查
df.isnull().sum()

City Name             0
Type               1712
Package               0
Variety               5
Sub Variety        1461
Grade              1757
Date                  0
Low Price             0
High Price            0
Mostly Low          103
Mostly High         103
Origin                3
Origin District    1626
Item Size           279
Color               616
Environment        1757
Unit of Sale       1595
Quality            1757
Condition          1757
Appearance         1757
Storage            1757
Crop               1757
Repack                0
Trans Mode         1757
Unnamed: 24        1757
Unnamed: 25        1654
dtype: int64

In [9]:
# 4.一致性检查
# 发现南瓜的包装方式（称量单位）不统一
print(df["Package"].is_unique)    # False
print(set(df["Package"].tolist()))

False
{'1 1/9 bushel crates', '50 lb cartons', 'bins', '1/2 bushel cartons', '40 lb cartons', '20 lb cartons', 'bushel baskets', '24 inch bins', '35 lb cartons', '50 lb sacks', '1 1/9 bushel cartons', 'each', 'bushel cartons', '22 lb cartons', '36 inch bins'}


In [10]:
# 提取本次研究所需要的有价值的特征和标签
features = ['Date', 'City Name', 'Variety', 'Package', 'Low Price', 'High Price']
data = df[features]
print(len(data))    # 1757
print(data.head().to_string())

1757
      Date  City Name      Variety       Package  Low Price  High Price
0  4/29/17  BALTIMORE          NaN  24 inch bins      270.0       280.0
1   5/6/17  BALTIMORE          NaN  24 inch bins      270.0       280.0
2  9/24/16  BALTIMORE  HOWDEN TYPE  24 inch bins      160.0       160.0
3  9/24/16  BALTIMORE  HOWDEN TYPE  24 inch bins      160.0       160.0
4  11/5/16  BALTIMORE  HOWDEN TYPE  24 inch bins       90.0       100.0


In [11]:
# 5.过滤缺失值
# 过滤含有空值的数据
data.dropna(axis=0, how='any', inplace=True)
print(len(data))     # 1752

1752


In [12]:
# 过滤“蒲式耳”单位称量的南瓜数据
data = data[data['Package'].str.contains('bushel', case=True, regex=True)]
print(len(data))     # 415
print(data.head().to_string())

415
       Date  City Name   Variety               Package  Low Price  High Price
70  9/24/16  BALTIMORE  PIE TYPE  1 1/9 bushel cartons       15.0        15.0
71  9/24/16  BALTIMORE  PIE TYPE  1 1/9 bushel cartons       18.0        18.0
72  10/1/16  BALTIMORE  PIE TYPE  1 1/9 bushel cartons       18.0        18.0
73  10/1/16  BALTIMORE  PIE TYPE  1 1/9 bushel cartons       17.0        17.0
74  10/8/16  BALTIMORE  PIE TYPE  1 1/9 bushel cartons       15.0        15.0


In [13]:
# 6.数据提取与转化
# 提取月份
data['Month'] = data['Date'].apply(lambda dt: pd.to_datetime(dt).month)
print(data.head().to_string())

# 销售日期转化为该年中的第几天
data['DayOfYear'] = data['Date'].apply(lambda dt: pd.to_datetime(dt).timetuple().tm_yday)
print(data.head().to_string())

# 计算南瓜的平均价格作为标签
data['Price'] = (data['Low Price'] + data['High Price']) / 2
print(data.head().to_string())

       Date  City Name   Variety               Package  Low Price  High Price  Month
70  9/24/16  BALTIMORE  PIE TYPE  1 1/9 bushel cartons       15.0        15.0      9
71  9/24/16  BALTIMORE  PIE TYPE  1 1/9 bushel cartons       18.0        18.0      9
72  10/1/16  BALTIMORE  PIE TYPE  1 1/9 bushel cartons       18.0        18.0     10
73  10/1/16  BALTIMORE  PIE TYPE  1 1/9 bushel cartons       17.0        17.0     10
74  10/8/16  BALTIMORE  PIE TYPE  1 1/9 bushel cartons       15.0        15.0     10
       Date  City Name   Variety               Package  Low Price  High Price  Month  DayOfYear
70  9/24/16  BALTIMORE  PIE TYPE  1 1/9 bushel cartons       15.0        15.0      9        268
71  9/24/16  BALTIMORE  PIE TYPE  1 1/9 bushel cartons       18.0        18.0      9        268
72  10/1/16  BALTIMORE  PIE TYPE  1 1/9 bushel cartons       18.0        18.0     10        275
73  10/1/16  BALTIMORE  PIE TYPE  1 1/9 bushel cartons       17.0        17.0     10        275
74  10/8/1

In [14]:
# 根据Package称量单位换算价格Price，转化为每bushel的价格（标准化）
# 1 1/9 bushel cartons => Price = Price/(1 + 1/9)
data.loc[data['Package'].str.contains('1 1/9'), 'Price'] = data['Price']/(1 + 1/9)
# 1/2 bushel cartons => Price = Price/(1/2)
data.loc[data['Package'].str.contains('1/2'), 'Price'] = data['Price']/(1/2)
print(data.head(30).to_string())

         Date  City Name    Variety               Package  Low Price  High Price  Month  DayOfYear   Price
70    9/24/16  BALTIMORE   PIE TYPE  1 1/9 bushel cartons       15.0        15.0      9        268  13.500
71    9/24/16  BALTIMORE   PIE TYPE  1 1/9 bushel cartons       18.0        18.0      9        268  16.200
72    10/1/16  BALTIMORE   PIE TYPE  1 1/9 bushel cartons       18.0        18.0     10        275  16.200
73    10/1/16  BALTIMORE   PIE TYPE  1 1/9 bushel cartons       17.0        17.0     10        275  15.300
74    10/8/16  BALTIMORE   PIE TYPE  1 1/9 bushel cartons       15.0        15.0     10        282  13.500
75    10/8/16  BALTIMORE   PIE TYPE  1 1/9 bushel cartons       18.0        18.0     10        282  16.200
76    10/8/16  BALTIMORE   PIE TYPE  1 1/9 bushel cartons       17.0        17.0     10        282  15.300
77    10/8/16  BALTIMORE   PIE TYPE  1 1/9 bushel cartons       17.0        18.5     10        282  15.975
78   10/15/16  BALTIMORE   PIE TYPE  

In [15]:
# 7.数据整理
# 将整理的数据放到新的DataFrame中
new_features = ['Month', 'DayOfYear', 'City Name', 'Variety', 'Package', 'Price']
new_pumpkins = data[new_features].reset_index(drop='index')
print(len(new_pumpkins))    # 共415条数据

# 删除包含空值的行
new_pumpkins.dropna(inplace=True)
print(len(new_pumpkins))    # 415
print(new_pumpkins.head().to_string())

415
415
   Month  DayOfYear  City Name   Variety               Package  Price
0      9        268  BALTIMORE  PIE TYPE  1 1/9 bushel cartons   13.5
1      9        268  BALTIMORE  PIE TYPE  1 1/9 bushel cartons   16.2
2     10        275  BALTIMORE  PIE TYPE  1 1/9 bushel cartons   16.2
3     10        275  BALTIMORE  PIE TYPE  1 1/9 bushel cartons   15.3
4     10        282  BALTIMORE  PIE TYPE  1 1/9 bushel cartons   13.5
